In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/software/'
import sys
import gc
# assuming data, models, engine in flicc directory:
flicc_path = os.path.join(os.path.dirname(os.getcwd()), '')
sys.path.append(flicc_path)
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

# import warnings
# warnings.filterwarnings("ignore")

In [2]:
best_config = {'bert-base-uncased':{'lr':5.0e-5},
                'roberta-large':{'lr':5.0e-5, 'focalloss':True, 'gamma':8},
                'gpt2':{'lr':5.0e-5},
                'bigscience/bloom-560m':{'lr':5.0e-5, 'focalloss':True, 'gamma':8},
                'facebook/opt-350m':{'lr':1.0e-5},
                'EleutherAI/gpt-neo-1.3B':{'lr':5.0e-5}, 
                'microsoft/deberta-base':{'lr':1.0e-5},
                'microsoft/deberta-v2-xlarge':{'lr':1.0e-5, 'focalloss':True, 'gamma':4}}

In [3]:
results = {'test_acc':[],
           'test_f1':[],
           'eval_acc':[],
           'eval_f1':[],
           'wd':[]}

In [4]:
weight_decay = [0.01, 0.1]

for model_checkpoint in best_config.keys():
    for wd in weight_decay:
        print(f'Grid search {model_checkpoint}, learning rate {best_config[model_checkpoint]["lr"]}')
        data = ClimateDataset(model_to_train=4,model_checkpoint=model_checkpoint,dataset_url=flicc_path,batch_size=32)
        data.setup_dataloaders()
        model = ClassificationModel(model_checkpoint=data.model_checkpoint,num_labels=data.num_labels)
        trainer = Engine(epochs=30,labels=data.labels)
        trainer.model = model.model
        trainer.dataset_encoded = data.dataset_encoded
        test_acc, test_f1, eval_acc, eval_f1 = trainer.run(**best_config[model_checkpoint],
                                                            wd=wd,
                                                            train_dataloader=data.train_dataloader,
                                                            eval_dataloader=data.eval_dataloader,
                                                            test_dataloader=data.test_dataloader,
                                                            early_stop=3)
        results['test_acc'].append(test_acc)
        results['test_f1'].append(test_f1)
        results['eval_acc'].append(eval_acc)
        results['eval_f1'].append(eval_f1)
        results['wd'].append(wd)
        print('### '*10)
        print(results)
        print('### '*10)
        with torch.no_grad():
            torch.cuda.empty_cache()
        del data, model, trainer
        gc.collect()

Grid search bert-base-uncased, learning rate 5e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	2.0826	Val Loss:	1.6531	Accuracy:	0.4989	F1:	0.3738 *
2 / 30: Train Loss:	1.2707	Val Loss:	1.3252	Accuracy:	0.5799	F1:	0.5194 *
3 / 30: Train Loss:	0.7319	Val Loss:	1.1258	Accuracy:	0.6324	F1:	0.5639 *
4 / 30: Train Loss:	0.3952	Val Loss:	1.4255	Accuracy:	0.5864	F1:	0.5423
5 / 30: Train Loss:	0.2623	Val Loss:	1.4625	Accuracy:	0.6236	F1:	0.5949 *
6 / 30: Train Loss:	0.1807	Val Loss:	1.3490	Accuracy:	0.6389	F1:	0.6072 *
7 / 30: Train Loss:	0.0986	Val Loss:	1.3551	Accuracy:	0.6324	F1:	0.5810
8 / 30: Train Loss:	0.0513	Val Loss:	1.3074	Accuracy:	0.6783	F1:	0.6467 *
9 / 30: Train Loss:	0.0241	Val Loss:	1.3179	Accuracy:	0.6849	F1:	0.6506 *
10 / 30: Train Loss:	0.0155	Val Loss:	1.3399	Accuracy:	0.6915	F1:	0.6668 *
11 / 30: Train Loss:	0.0142	Val Loss:	1.3720	Accuracy:	0.7024	F1:	0.6720 *
12 / 30: Train Loss:	0.0082	Val Loss:	1.4112	Accuracy:	0.6958	F1:	0.6679
13 / 30: Train Loss:	0.0067	Val Loss:	1.4371	Accuracy:	0.6915	F1:	0.6635
14 / 30: Train Loss:	0.0058	Val Loss:	1.46

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	2.0645	Val Loss:	1.5972	Accuracy:	0.5098	F1:	0.4014 *
2 / 30: Train Loss:	1.2741	Val Loss:	1.2860	Accuracy:	0.5733	F1:	0.4912 *
3 / 30: Train Loss:	0.7838	Val Loss:	1.1558	Accuracy:	0.6346	F1:	0.5662 *
4 / 30: Train Loss:	0.4551	Val Loss:	1.2947	Accuracy:	0.6346	F1:	0.5817 *
5 / 30: Train Loss:	0.2872	Val Loss:	1.1718	Accuracy:	0.6761	F1:	0.6286 *
6 / 30: Train Loss:	0.1453	Val Loss:	1.2508	Accuracy:	0.6630	F1:	0.6413 *
7 / 30: Train Loss:	0.0624	Val Loss:	1.3152	Accuracy:	0.6849	F1:	0.6613 *
8 / 30: Train Loss:	0.0404	Val Loss:	1.3535	Accuracy:	0.6586	F1:	0.6180
9 / 30: Train Loss:	0.0338	Val Loss:	1.3915	Accuracy:	0.6652	F1:	0.6335
10 / 30: Train Loss:	0.0336	Val Loss:	1.5391	Accuracy:	0.6346	F1:	0.6125
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem       0.86      0.57      0.68        67
               anecdote       0.97      0.84  

...

1 / 30: Train Loss:	2.3659	Val Loss:	2.0159	Accuracy:	0.3479	F1:	0.2233 *
2 / 30: Train Loss:	1.7853	Val Loss:	1.5334	Accuracy:	0.4967	F1:	0.3598 *
3 / 30: Train Loss:	1.3402	Val Loss:	1.3696	Accuracy:	0.5558	F1:	0.4308 *
4 / 30: Train Loss:	1.0260	Val Loss:	1.2266	Accuracy:	0.6061	F1:	0.4942 *
5 / 30: Train Loss:	0.7902	Val Loss:	1.1374	Accuracy:	0.6368	F1:	0.5711 *
6 / 30: Train Loss:	0.6133	Val Loss:	1.1232	Accuracy:	0.6565	F1:	0.6209 *
7 / 30: Train Loss:	0.4791	Val Loss:	1.1839	Accuracy:	0.6477	F1:	0.6259 *
8 / 30: Train Loss:	0.3811	Val Loss:	1.3164	Accuracy:	0.6368	F1:	0.6161
9 / 30: Train Loss:	0.2981	Val Loss:	1.2401	Accuracy:	0.6696	F1:	0.6561 *
10 / 30: Train Loss:	0.2376	Val Loss:	1.2508	Accuracy:	0.6718	F1:	0.6583 *
11 / 30: Train Loss:	0.1918	Val Loss:	1.2296	Accuracy:	0.6827	F1:	0.6846 *
12 / 30: Train Loss:	0.1465	Val Loss:	1.3557	Accuracy:	0.6521	F1:	0.6464
13 / 30: Train Loss:	0.1139	Val Loss:	1.3257	Accuracy:	0.6761	F1:	0.6716
14 / 30: Train Loss:	0.0882	Val Loss:	1.

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	21.0389	Val Loss:	2.1833	Accuracy:	0.2932	F1:	0.2272 *
2 / 30: Train Loss:	11.5078	Val Loss:	1.3374	Accuracy:	0.5777	F1:	0.5519 *
3 / 30: Train Loss:	4.9419	Val Loss:	1.1115	Accuracy:	0.6543	F1:	0.6404 *
4 / 30: Train Loss:	2.4948	Val Loss:	0.9952	Accuracy:	0.6674	F1:	0.6597 *
5 / 30: Train Loss:	1.2027	Val Loss:	0.9343	Accuracy:	0.7155	F1:	0.7126 *
6 / 30: Train Loss:	0.6009	Val Loss:	0.8813	Accuracy:	0.7155	F1:	0.7099
7 / 30: Train Loss:	0.3988	Val Loss:	0.9104	Accuracy:	0.7068	F1:	0.7012
8 / 30: Train Loss:	0.2287	Val Loss:	0.8626	Accuracy:	0.7199	F1:	0.7183 *
9 / 30: Train Loss:	0.1779	Val Loss:	0.8560	Accuracy:	0.7177	F1:	0.7072
10 / 30: Train Loss:	0.0951	Val Loss:	0.8309	Accuracy:	0.7352	F1:	0.7256 *
11 / 30: Train Loss:	0.0695	Val Loss:	0.8607	Accuracy:	0.7330	F1:	0.7156
12 / 30: Train Loss:	0.0662	Val Loss:	0.8494	Accuracy:	0.7243	F1:	0.7170
13 / 30: Train Loss:	0.0421	Val Loss:	0.8227	Accuracy:	0.7330	F1:	0.7278 *
14 / 30: Train Loss:	0.0205	Val Loss:	0.82

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	20.8805	Val Loss:	2.0141	Accuracy:	0.3348	F1:	0.2547 *
2 / 30: Train Loss:	10.2292	Val Loss:	1.2911	Accuracy:	0.6193	F1:	0.5923 *
3 / 30: Train Loss:	4.2030	Val Loss:	1.1167	Accuracy:	0.6389	F1:	0.6179 *
4 / 30: Train Loss:	1.9775	Val Loss:	1.0339	Accuracy:	0.6499	F1:	0.6522 *
5 / 30: Train Loss:	1.1588	Val Loss:	0.9071	Accuracy:	0.7046	F1:	0.7013 *
6 / 30: Train Loss:	0.5947	Val Loss:	0.9418	Accuracy:	0.6980	F1:	0.6912
7 / 30: Train Loss:	0.6196	Val Loss:	0.9151	Accuracy:	0.7155	F1:	0.6972
8 / 30: Train Loss:	0.3262	Val Loss:	0.9448	Accuracy:	0.7002	F1:	0.6899
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem       0.74      0.69      0.71        67
               anecdote       0.95      0.86      0.90        43
         cherry picking       0.60      0.79      0.68        56
      conspiracy theory       0.79      0.69      0.74        3

In [5]:
import pandas as pd

In [6]:
pd.DataFrame(results)

,test_acc,test_f1,eval_acc,eval_f1,wd
0,0.652344,0.636726,0.702407,0.671951,0.01
1,0.632812,0.624441,0.684902,0.661278,0.10
2,0.031250,0.005051,0.030635,0.004954,0.01
3,0.027344,0.004436,0.028446,0.004610,0.10
4,0.601562,0.570647,0.582057,0.577425,0.01
5,0.539062,0.504938,0.555799,0.533732,0.10
6,0.488281,0.464992,0.509847,0.508219,0.01
7,0.507812,0.511482,0.520788,0.523660,0.10
8,0.296875,0.213467,0.288840,0.196232,0.01
9,0.300781,0.215178,0.280088,0.187681,0.10
